In [9]:
import os
from typing import Any, List

from langchain_community.chat_models import ChatLiteLLM
from langchain_core.agents import AgentAction, AgentFinish
from langchain_core.output_parsers import BaseOutputParser
from langchain_core.output_parsers.base import T
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.agents import Tool

from langchain.agents.agent import AgentOutputParser

In [3]:
def search(query):
    # This function should implement the logic to search for products
    # For now, it just returns a placeholder response
    return f"Searching for: {query}"

tools = [
    Tool(
            name="ProductSearch",
            func=search,
            description="useful for when you need to answer questions about product information or services offered, availability and their costs.",
        )
]

In [5]:
class CustomPromptTemplateForTools(PromptTemplate):

    template: str
    tools: list[Tool]

    def format(self, **kwargs: Any) -> str:
        kwargs["tools"] = "\n".join(
            [f"{tool.name}: {tool.description}" for tool in tools]
        )
        return self.template.format(**kwargs)

prompt = CustomPromptTemplateForTools(
    template="You are an AI Sales agent, sell me anything，Here are the questions you must answer:{input}\n\nAnd you has access to the following tools: \n{tools} \n{intermediate_steps}",
    tools=tools,
    input_variables=["input", "intermediate_steps"],
)
print(prompt.format(input="Who are you?", intermediate_steps=[]))

You are an AI Sales agent, sell me anything，Here are the questions you must answer:Who are you?

And you has access to the following tools: 
ProductSearch: useful for when you need to answer questions about product information or services offered, availability and their costs. 
[]


In [ ]:
os.environ["DASHSCOPE_API_KEY"] = 'sk-36c7f4e71d904a35afab94dc68747d48'
llm = ChatLiteLLM(model="qwen-max",
                  api_base="https://dashscope.aliyuncs.com/compatible-mode/v1",
                  api_key=os.environ.get('DASHSCOPE_API_KEY'))

In [ ]:
llm_chain = LLMChain(llm=llm, prompt=prompt, verbose=True)

In [13]:
class SalesConvoOutputParser(BaseOutputParser):

    def parse(self, text: str) -> List[str]:
        # This function should implement the logic to parse the output
        # For now, it just returns the text as a list
        return text.split(",")



output_parser = SalesConvoOutputParser()
outRest = output_parser.invoke(input="Hello, my name is John")
print(outRest)

['Hello', ' my name is John']


In [ ]:
sales_agent_with_tools = LLMSingleActionAgent(
                llm_chain=llm_chain,
                output_parser=output_parser,
                stop=["\nObservation:"],
                allowed_tools=tool_names,
            )